In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [23]:
# Load the data
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

# Assign X and Y from training data
X_train = train.drop(labels = ["label"],axis = 1) 
y_train = train["label"].values

# Normalize the data for better performance
X_train = X_train / 255.0
test = test / 255.0

# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)

In [24]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [27]:
batch_size = 128
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/20
296/296 [==============================] - 14s 48ms/step - loss: 0.0937 - accuracy: 0.9708 - val_loss: 0.0575 - val_accuracy: 0.9814
Epoch 2/20
296/296 [==============================] - 16s 53ms/step - loss: 0.0774 - accuracy: 0.9758 - val_loss: 0.0513 - val_accuracy: 0.9848
Epoch 3/20
296/296 [==============================] - 15s 49ms/step - loss: 0.0633 - accuracy: 0.9804 - val_loss: 0.0470 - val_accuracy: 0.9850
Epoch 4/20
296/296 [==============================] - 16s 55ms/step - loss: 0.0588 - accuracy: 0.9819 - val_loss: 0.0395 - val_accuracy: 0.9871
Epoch 5/20
296/296 [==============================] - 16s 56ms/step - loss: 0.0523 - accuracy: 0.9836 - val_loss: 0.0411 - val_accuracy: 0.9874
Epoch 6/20
296/296 [==============================] - 15s 50ms/step - loss: 0.0466 - accuracy: 0.9856 - val_loss: 0.0402 - val_accuracy: 0.9871
Epoch 7/20
296/296 [==============================] - 15s 50ms/step - loss: 0.0469 - accuracy: 0.9849 - val_loss: 0.0342 - val_accuracy:

In [28]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)